# 0. Development Setting

In [156]:
# !pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 47.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 28.4 MB/s eta 0:00:0000:0100:01


In [2]:
!python -V

Python 3.9.16


In [3]:
!pip list

Package                      Version
---------------------------- ---------
absl-py                      1.4.0
anyio                        3.6.2
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
arrow                        1.2.3
asttokens                    2.2.1
astunparse                   1.6.3
attrs                        22.2.0
backcall                     0.2.0
beautifulsoup4               4.11.2
bleach                       6.0.0
boto3                        1.26.72
botocore                     1.29.72
cachetools                   5.3.0
certifi                      2022.12.7
cffi                         1.15.1
charset-normalizer           3.0.1
comm                         0.1.2
debugpy                      1.6.6
decorator                    5.1.1
defusedxml                   0.7.1
easydict                     1.10
executing                    1.2.0
fastjsonschema               2.16.2
flatbuffers                  23.1.21
fqdn                         1.5.1

In [111]:
import pandas as pd
import tensorflow as tf
import pymysql
import boto3
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from smart_open import open as s_open
from dotenv import load_dotenv
from easydict import EasyDict

from datetime import datetime
import time
import os

load_dotenv()

True

In [2]:
settings = EasyDict()

settings.AWS_ACCESS_KEY_ID = os.environ['AWS_ACCESS_KEY_ID']     
settings.AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]
settings.AWS_REGION_NAME = os.environ["AWS_REGION_NAME"]   
settings.AWS_ACCOUNT_ID = os.environ["AWS_ACCOUNT_ID"]
settings.AWS_BUCKET_NAME = 'genia-bucket'
settings.AWS_ATHENA_OUTPUT_LOCATION = 'raw/student1/output'
settings.AWS_ATHENA_DATABASE = 'mini_db'

In [3]:
class Boto3Client(object):
    aws_access_key_id = settings.AWS_ACCESS_KEY_ID
    aws_secret_access_key = settings.AWS_SECRET_ACCESS_KEY
    region_name = settings.AWS_REGION_NAME
    bucket_name = settings.AWS_BUCKET_NAME
    athena_database = settings.AWS_ATHENA_DATABASE
    athena_output_location = settings.AWS_ATHENA_OUTPUT_LOCATION
    
    service_name = None
    
    @classmethod
    def get_client(cls):
        options = dict(
            aws_access_key_id = Boto3Client.aws_access_key_id,
            aws_secret_access_key = Boto3Client.aws_secret_access_key,
            region_name = Boto3Client.region_name 
        )
        
        return boto3.client(cls.service_name, **options)
        # 위와 같다.
#         return boto3.client(cls.service_name,
#                             aws_access_key_id = Boto3Client.aws_access_key_id,
#                             aws_secret_access_key = Boto3Client.aws_secret_access_key,
#                             region_name = Boto3Client.region_name)


* @classmethod
    * 상속받아서 쓰는 클래스에서 해당 내용을 덮어쓰기
* @staticmethod
    * 그대로 .. 쓴다. ?

In [105]:
# 아래 두 class는 위의 Boto3Client 의 내용을 그대로 상속받는다.
# 그런데 그 중에서 아래에 표시된 service_name 부분만 해당 문자열로 바뀐다. 
import io

class S3Client(Boto3Client):
    
    service_name = 's3'
    
    
    @staticmethod
    def get_s3_df(file_name: str):
        clnt = S3Client.get_client()
        obj = clnt.get_object(
                                Bucket = S3Client.bucket_name,
                                Key = file_name
                              )
        
        return pd.read_csv(obj["Body"], encoding = 'utf-8')
    
    @staticmethod
    def get_s3_parquet_df(file_name: str):
        clnt = S3Client.get_client()
        obj = clnt.get_object(
                                Bucket = S3Client.bucket_name,
                                Key = file_name
                              )
        return pd.read_parquet(io.BytesIO(obj["Body"].read()), engine="pyarrow")
        
    
    
    @staticmethod
    def upload_s3_df(df:pd.DataFrame, file_name: str):
        try:
            clnt = S3Client.get_client()
            file_name = f"s3://{S3Client.bucket_name}/{file_name}"
            with s_open(file_name, "wb", transport_params = dict(client = clnt)) as out_file:
                df.to_parquet(out_file, engine = 'pyarrow', compression = 'gzip', index = False)

            return True

        except Exception as e:
            print(str(e))
            return False
    


In [6]:
# 경로를 설정할 때 / 표시에 주의하자. 실수로 한번 더 입력하거나 중복 입력하게 되는 순간 엉뚱한 곳에서 찾는다.
# 이걸로 2시간을 헤매고 다녔다.

class AthenaClient(Boto3Client):
    
    service_name = "athena"
    output_location = f's3://{Boto3Client.bucket_name}/{Boto3Client.athena_output_location}'
    
    @staticmethod
    def get_athena_query_exec_id(sql: str):
        
        clnt = AthenaClient.get_client()
        response = clnt.start_query_execution(
                QueryString=sql,
                QueryExecutionContext={
                    'Database': AthenaClient.athena_database
                },
                ResultConfiguration={
                    'OutputLocation': AthenaClient.output_location,
                }
            )

        return response['QueryExecutionId']
    
    @staticmethod
    def collect_query_result(query_execution_id):
        
        clnt = AthenaClient.get_client()
        
        WAIT = ['QUEUED', 'RUNNING']
        SUCCESS = ['SUCCEEDED']
        FAILED = ['FAILED', 'CANCELLED']

        while True:

            try:
                query_status = clnt.get_query_execution(QueryExecutionId=query_execution_id)
                query_execution_status = query_status['QueryExecution']['Status']['State']

                if query_execution_status in SUCCESS:
                    print('SUCCESSED!')
                    query_result_path = f'{AthenaClient.output_location}/{query_execution_id}.csv'
                    query_result_path = query_result_path.replace(f"s3://{AthenaClient.bucket_name}/", "")
                    return S3Client.get_s3_df(query_result_path)

                if query_execution_status in FAILED:
                    print(f'FAILED!! -> {query_execution_status}')
                    break

                if query_execution_status in WAIT:
                    print(f'Still Running... -> {query_execution_status}')
                    time.sleep(0.5)
                    continue

                print(f'unexpected status!! -> {query_execution_status}')
                break

            except Exception as e:
                print(str(e))
                break
    
        return False
    
    @staticmethod
    def get_athena_sql(sql: str):
        query_exec_id = AthenaClient.get_athena_query_exec_id(sql)
        print(query_exec_id)
        return AthenaClient.collect_query_result(query_exec_id)
    
    


In [71]:
sample_sql = '''SELECT * 
                FROM "student1-hotel" 
                WHERE 1=1
                    AND arrival_date_year = 2017 
                    AND arrival_date_month = 'February' 
                LIMIT 10;'''
result = AthenaClient.get_athena_sql(sample_sql)

result

1a6f8bec-fbbe-473f-8f84-523e31316e92
Still Running... -> QUEUED
Still Running... -> RUNNING
Still Running... -> RUNNING
SUCCESSED!


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,yyyy,mm
0,Resort Hotel,1,76,2017,February,5,1,0,1,2,...,NaN,0,Transient,42.0,0,1,Canceled,2016-11-21,2017,2
1,Resort Hotel,1,11,2017,February,5,1,0,2,2,...,NaN,0,Transient,48.0,0,0,Canceled,2017-01-21,2017,2
2,Resort Hotel,1,0,2017,February,5,1,2,6,2,...,NaN,0,Transient,59.0,0,0,Canceled,2017-02-01,2017,2
3,Resort Hotel,1,123,2017,February,5,1,4,10,2,...,NaN,0,Transient,37.8,0,0,Canceled,2016-12-24,2017,2
4,Resort Hotel,1,85,2017,February,5,1,0,4,2,...,NaN,0,Transient,72.0,0,1,Canceled,2016-11-13,2017,2
5,Resort Hotel,1,0,2017,February,5,2,0,2,2,...,NaN,0,Transient,55.0,0,0,No-Show,2017-02-02,2017,2
6,Resort Hotel,1,2,2017,February,5,2,0,2,2,...,NaN,0,Transient,48.0,0,0,Canceled,2017-01-31,2017,2
7,Resort Hotel,1,169,2017,February,5,2,0,3,2,...,NaN,0,Transient,37.8,0,2,Canceled,2016-11-13,2017,2
8,Resort Hotel,1,169,2017,February,5,2,0,3,2,...,NaN,0,Transient,37.8,0,3,Canceled,2016-11-13,2017,2
9,Resort Hotel,1,81,2017,February,5,2,0,3,1,...,NaN,0,Transient-Party,37.0,0,1,Canceled,2017-01-14,2017,2


# 1. Load Dataset (Amazon Athena)

In [7]:
sample_sql = '''SELECT * 
                FROM "student1-hotel" 
                ;'''
result = AthenaClient.get_athena_sql(sample_sql)

result

7c72f4da-abf8-4a13-bb95-84ad1c7d3405
Still Running... -> QUEUED
Still Running... -> RUNNING
Still Running... -> RUNNING
Still Running... -> RUNNING
Still Running... -> RUNNING
SUCCESSED!


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,yyyy,mm
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01,2015,7
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01,2015,7
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02,2015,7
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02,2015,7
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03,2015,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114460,City Hotel,0,115,2017,April,18,30,2,4,2,...,NaN,0,Transient,138.00,0,2,Check-Out,2017-05-06,2017,4
114461,City Hotel,0,195,2017,April,17,29,2,5,2,...,NaN,0,Transient,95.46,0,1,Check-Out,2017-05-06,2017,4
114462,City Hotel,0,12,2017,April,18,30,2,4,3,...,NaN,0,Transient,206.47,0,1,Check-Out,2017-05-06,2017,4
114463,City Hotel,0,10,2017,April,17,26,2,9,1,...,NaN,0,Transient,19.09,0,3,Check-Out,2017-05-07,2017,4


# 2. Preprocessing (EDA)
* market_segment, distribution_channel 컬럼 반드시 포함
* reservation_status 컬럼 포함하지 않을 것
* agent, company 정보는 null값이 너무 많아서 빼고 진행하기로 결정했음
* country 정보도 살려둘까 했지만. 새로 test하는 데이터에서 문제가 생기므로, 해당 컬럼도 빼고 진행하기로 결정했음
* hotel 컬럼은 빼고 진행했음

In [11]:
# 모델 학습 시 컨트리 정보도 쓰지 않기로 했으므로 삭제
# result['country'] = result['country'].fillna('no')

In [8]:
# children 정보가 없다...는 것은
# 아이들이 없었다고 생각하기로 한다. 
# 예약정보를 받아오는 것이라면 누락시켜서 표기했을 가능성이 크다.
# 해당 컬럼은 float64 형태이므로 0.0으로 넣기로 한다.
result['children'] = result['children'].fillna(0.0)

In [9]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114465 entries, 0 to 114464
Data columns (total 34 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           114465 non-null  object 
 1   is_canceled                     114465 non-null  int64  
 2   lead_time                       114465 non-null  int64  
 3   arrival_date_year               114465 non-null  int64  
 4   arrival_date_month              114465 non-null  object 
 5   arrival_date_week_number        114465 non-null  int64  
 6   arrival_date_day_of_month       114465 non-null  int64  
 7   stays_in_weekend_nights         114465 non-null  int64  
 8   stays_in_week_nights            114465 non-null  int64  
 9   adults                          114465 non-null  int64  
 10  children                        114465 non-null  float64
 11  babies                          114465 non-null  int64  
 12  meal            

In [10]:
# 예약 변경 '날짜'만 보기 위해 작업
# 이 컬럼은 원래 '2010-02-09' 와 같이 연월일이 전부 표기되어 있다.
# 컬럼 중 날짜정보가 있는 컬럼이 없기도 하고, 이 정보를 범주형변수로 보고 원핫인코딩 했을 시 컬럼만 엄청 늘어날 것이다.
# 뒤에 yyyy, mm 컬럼을 통해 연 월 정보를 알 수 있으므로, 다른 정보는 전부 버리고 일자만 남기기로 했다.

result['reservation_status_date'] = result['reservation_status_date'].apply(lambda x : str(x[-2:]))

In [11]:
# 아래는 컬럼들을 수치형, 범주형으로 나눠주고, y도 분리해주기 위해 작성했다.
# reservation_status 삭제
# 기존에 들어 있던 country 역시 삭제.
# int, float라고 꼭 수치형은 아닐 것이므로, int, float인 컬럼 중에서도 범주형이라고 생각되는 컬럼은 범주형에 넣었다.

numerical_columns = ['lead_time',
                     'stays_in_weekend_nights',
                     'stays_in_week_nights',
                     'adults',
                     'children',
                     'babies',
                     'previous_bookings_not_canceled',
                     'previous_cancellations',
                     'booking_changes',
                     'days_in_waiting_list',
                     'adr',
                     'required_car_parking_spaces',
                     'total_of_special_requests']

categorical_columns = ['arrival_date_year',
                       'arrival_date_month',
                       'arrival_date_week_number',
                       'arrival_date_day_of_month',
                       'meal',
                       'market_segment',
                       'distribution_channel',
                       'is_repeated_guest',
                       'reserved_room_type',
                       'assigned_room_type',
                       'deposit_type',
                       'customer_type',
                       'yyyy',
                       'mm',
                       'reservation_status_date']

y_columns = "is_canceled"

In [12]:
# 위에 작성한 컬럼 정보들을 활용해 데이터프레임을 나눠 보기로 한다. 

numerical_df = result[numerical_columns]
categorical_df = result[categorical_columns]
y_data = result[y_columns]

### 범주형 변수 원핫인코딩

In [13]:
# 우선 범주형 변수이다.
# get_dummies를 쓰기 위해서는 해당 컬럼이 스트링 값이어야 하는 것 같다.
# 컬럼 중 숫자로 된 정보도 있었기 때문에 모든 컬럼을 .astype('str') 해 준다.
categorical_df = categorical_df.astype('str')

In [14]:
cate_encoded_df = pd.get_dummies(categorical_df)

In [15]:
# 처리했더니 총 194개의 컬럼이 만들어졌다.
cate_encoded_df

,arrival_date_year_2015,arrival_date_year_2016,arrival_date_year_2017,arrival_date_month_April,arrival_date_month_August,arrival_date_month_December,arrival_date_month_February,arrival_date_month_January,arrival_date_month_July,arrival_date_month_June,...,reservation_status_date_22,reservation_status_date_23,reservation_status_date_24,reservation_status_date_25,reservation_status_date_26,reservation_status_date_27,reservation_status_date_28,reservation_status_date_29,reservation_status_date_30,reservation_status_date_31
0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114460,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114461,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114462,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114463,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### X_data 만들기

* 위에서 만든 범주형 변수의 194개 컬럼과, 수치형 변수 데이터프레임을 이어붙여 X_data 데이터프레임을 만든다.

In [16]:
X_data = pd.concat([numerical_df, cate_encoded_df], axis = 1)

### X_data 정규화
* StandardSclaer를 통해 해당 데이터를 정규화 해 주기로 한다.

In [17]:
std_scl = StandardScaler()

In [18]:
# fit으로 정규분포의 평균, 분산 등을 정해 준 후 
# transform으로 값을 변경해준다.
# 둘을 한번에 해 주는 것이 fit_transform 이다.
# 여기서부터 X_data 는 numpy.ndarray 로 바뀌어 있다.

X_data = std_scl.fit_transform(X_data)

In [19]:
# 컬럼은 총 207개다.
X_data.shape, y_data.shape

((114465, 207), (114465,))

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data,
    y_data,
    stratify = y_data,
    test_size = 0.25,
    random_state = 42)

In [21]:
X_train.shape

(85848, 207)

# 3. DeepLearning Model Architecture

In [22]:
# is_canceled 가 캔슬 여부 0, 1 로 표기된 바이너리 형태이다.
# 그러므로 마지막은 1의 sigmoid 로 주어야 한다.
# 예전에 타이타닉 예측 시 활용했던 모델을 그대로 가져와 보았다.

model = Sequential()

model.add(keras.Input(shape=X_train.shape[1:]))

model.add(layers.Dense(128))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

model.add(layers.Dropout(0.2))

model.add(layers.Dense(64))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

model.add(layers.Dropout(0.1))

model.add(layers.Dense(1, activation = 'sigmoid'))
          
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               26624     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                        

2023-02-21 01:17:25.420414: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-21 01:17:25.420455: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-21 01:17:25.420480: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2acbb51949a0): /proc/driver/nvidia/version does not exist
2023-02-21 01:17:25.420721: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
# loss 는 binary_crossentropy 로 쓴다.

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

# 4. Train Model

In [24]:
# 모델 학습.
history = model.fit(
    X_train,
    y_train,
    validation_split=.2,
    epochs=30,
    batch_size=500,
    verbose=2,
)

Epoch 1/30
138/138 - 3s - loss: 0.4772 - accuracy: 0.7670 - val_loss: 0.3650 - val_accuracy: 0.8413 - 3s/epoch - 21ms/step
Epoch 2/30
138/138 - 1s - loss: 0.3407 - accuracy: 0.8522 - val_loss: 0.2568 - val_accuracy: 0.9034 - 1s/epoch - 7ms/step
Epoch 3/30
138/138 - 1s - loss: 0.2425 - accuracy: 0.9052 - val_loss: 0.1896 - val_accuracy: 0.9373 - 977ms/epoch - 7ms/step
Epoch 4/30
138/138 - 1s - loss: 0.1933 - accuracy: 0.9272 - val_loss: 0.1556 - val_accuracy: 0.9457 - 1s/epoch - 7ms/step
Epoch 5/30
138/138 - 1s - loss: 0.1660 - accuracy: 0.9382 - val_loss: 0.1363 - val_accuracy: 0.9519 - 948ms/epoch - 7ms/step
Epoch 6/30
138/138 - 1s - loss: 0.1469 - accuracy: 0.9469 - val_loss: 0.1230 - val_accuracy: 0.9564 - 1s/epoch - 8ms/step
Epoch 7/30
138/138 - 1s - loss: 0.1301 - accuracy: 0.9524 - val_loss: 0.1128 - val_accuracy: 0.9618 - 998ms/epoch - 7ms/step
Epoch 8/30
138/138 - 1s - loss: 0.1193 - accuracy: 0.9578 - val_loss: 0.1040 - val_accuracy: 0.9658 - 1s/epoch - 7ms/step
Epoch 9/30
138

# 5. Evaluate Model & Hyperparameter Tuning

In [25]:
# 만들어 둔 test셋으로 확인해 본 결과 정확도는 0.98 정도이다.
model.evaluate(X_test, y_test)

895/895 [==============================] - 1s 1ms/step - loss: 0.0582 - accuracy: 0.9836


[0.058236781507730484, 0.9836111664772034]

# 6. Inference Model & Upload S3

In [26]:
# 진짜 예측해야 할 데이터 정보.
sample_sql = 'SELECT * FROM "mini_db"."test-test_hotel";'

test_df = AthenaClient.get_athena_sql(sample_sql)
test_df

4fa45fd7-fb61-4f55-ad3c-2ae5a1f4f406
Still Running... -> QUEUED
Still Running... -> RUNNING
SUCCESSED!


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,82,2017,August,35,29,0,3,2,...,No Deposit,240.0,NaN,0,Transient,230.00,0,3,Check-Out,2017-09-01
1,Resort Hotel,0,42,2017,August,34,23,2,7,2,...,No Deposit,314.0,NaN,0,Transient,205.18,0,0,Check-Out,2017-09-01
2,Resort Hotel,0,128,2017,August,34,25,2,5,2,...,No Deposit,240.0,NaN,0,Transient,308.57,0,2,Check-Out,2017-09-01
3,Resort Hotel,0,37,2017,August,34,26,2,4,2,...,No Deposit,250.0,NaN,0,Transient,277.67,1,1,Check-Out,2017-09-01
4,Resort Hotel,0,21,2017,August,34,25,2,5,2,...,No Deposit,240.0,NaN,0,Transient,277.71,1,3,Check-Out,2017-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4920,City Hotel,0,23,2017,August,35,30,2,5,2,...,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06
4921,City Hotel,0,102,2017,August,35,31,2,5,3,...,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07
4922,City Hotel,0,34,2017,August,35,31,2,5,2,...,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07
4923,City Hotel,0,109,2017,August,35,31,2,5,2,...,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07


In [166]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4925 entries, 0 to 4924
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           4925 non-null   object 
 1   is_canceled                     4925 non-null   int64  
 2   lead_time                       4925 non-null   int64  
 3   arrival_date_year               4925 non-null   int64  
 4   arrival_date_month              4925 non-null   object 
 5   arrival_date_week_number        4925 non-null   int64  
 6   arrival_date_day_of_month       4925 non-null   int64  
 7   stays_in_weekend_nights         4925 non-null   int64  
 8   stays_in_week_nights            4925 non-null   int64  
 9   adults                          4925 non-null   int64  
 10  children                        4925 non-null   float64
 11  babies                          4925 non-null   int64  
 12  meal                            49

In [27]:
# yyyy, mm 생성
# mm 생성 시 기존 str(x[5:7]) 에서 추후 컬럼 생성 시 문제가 발생했으므로, 아래와 같이 수정

test_df['yyyy'] = test_df['reservation_status_date'].apply(lambda x : str(x[:4]))
test_df['mm'] = test_df['reservation_status_date'].apply(lambda x : str(int(x[5:7])))

In [28]:
test_df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,yyyy,mm
0,Resort Hotel,0,82,2017,August,35,29,0,3,2,...,NaN,0,Transient,230.00,0,3,Check-Out,2017-09-01,2017,9
1,Resort Hotel,0,42,2017,August,34,23,2,7,2,...,NaN,0,Transient,205.18,0,0,Check-Out,2017-09-01,2017,9
2,Resort Hotel,0,128,2017,August,34,25,2,5,2,...,NaN,0,Transient,308.57,0,2,Check-Out,2017-09-01,2017,9
3,Resort Hotel,0,37,2017,August,34,26,2,4,2,...,NaN,0,Transient,277.67,1,1,Check-Out,2017-09-01,2017,9
4,Resort Hotel,0,21,2017,August,34,25,2,5,2,...,NaN,0,Transient,277.71,1,3,Check-Out,2017-09-01,2017,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4920,City Hotel,0,23,2017,August,35,30,2,5,2,...,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06,2017,9
4921,City Hotel,0,102,2017,August,35,31,2,5,3,...,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07,2017,9
4922,City Hotel,0,34,2017,August,35,31,2,5,2,...,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07,2017,9
4923,City Hotel,0,109,2017,August,35,31,2,5,2,...,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07,2017,9


In [29]:
# reservation_status_date 에서 날짜만 추출해 넣기
test_df['reservation_status_date'] = test_df['reservation_status_date'].apply(lambda x : str(x[-2:]))

In [30]:
# 내용은 위에서 나눴던 것과 같습니다.
numerical_columns = ['lead_time',
                     'stays_in_weekend_nights',
                     'stays_in_week_nights',
                     'adults',
                     'children',
                     'babies',
                     'previous_bookings_not_canceled',
                     'previous_cancellations',
                     'booking_changes',
                     'days_in_waiting_list',
                     'adr',
                     'required_car_parking_spaces',
                     'total_of_special_requests']
categorical_columns = ['arrival_date_year',
                       'arrival_date_month',
                       'arrival_date_week_number',
                       'arrival_date_day_of_month',
                       'meal',
                       'market_segment',
                       'distribution_channel',
                       'is_repeated_guest',
                       'reserved_room_type',
                       'assigned_room_type',
                       'deposit_type',
                       'customer_type',
                       'yyyy',
                       'mm',
                       'reservation_status_date']
y_columns = "is_canceled"

In [31]:
test_numerical_df = test_df[numerical_columns]
test_categorical_df = test_df[categorical_columns]
test_y_data = test_df[y_columns]

### 범주형 변수 처리

In [32]:
test_categorical_df = test_categorical_df.astype('str')

In [33]:
test_cate_encoded_df = pd.get_dummies(test_categorical_df)

In [34]:
test_cate_encoded_df

,arrival_date_year_2017,arrival_date_month_August,arrival_date_week_number_31,arrival_date_week_number_32,arrival_date_week_number_33,arrival_date_week_number_34,arrival_date_week_number_35,arrival_date_day_of_month_1,arrival_date_day_of_month_10,arrival_date_day_of_month_11,...,reservation_status_date_22,reservation_status_date_23,reservation_status_date_24,reservation_status_date_25,reservation_status_date_26,reservation_status_date_27,reservation_status_date_28,reservation_status_date_29,reservation_status_date_30,reservation_status_date_31
0,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4920,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4921,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4922,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4923,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* cate_encoded_df 와 test_cate_encoded_df 의 컬럼을 보면 갯수가 다르다는 것을 알 수 있다.
* get_dummies 를 했을 때 포함된 인자들의 수가 적었을 것... 그렇다면?
* 기존 cate_encoded_df에는 있었지만 test_cate_encoded_df 에는 없는 컬럼들을 임의적으로 추가해 주어야 할 것 같다.
* 일단 어떤 컬럼들에서 차이가 있는지 확인 해 보자. 

In [35]:
# 기존 범주형 데이터프레임
cate_encoded_df.head()

,arrival_date_year_2015,arrival_date_year_2016,arrival_date_year_2017,arrival_date_month_April,arrival_date_month_August,arrival_date_month_December,arrival_date_month_February,arrival_date_month_January,arrival_date_month_July,arrival_date_month_June,...,reservation_status_date_22,reservation_status_date_23,reservation_status_date_24,reservation_status_date_25,reservation_status_date_26,reservation_status_date_27,reservation_status_date_28,reservation_status_date_29,reservation_status_date_30,reservation_status_date_31
0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# 테스트로 받았던 범주형 데이터프레임
test_cate_encoded_df.head()

,arrival_date_year_2017,arrival_date_month_August,arrival_date_week_number_31,arrival_date_week_number_32,arrival_date_week_number_33,arrival_date_week_number_34,arrival_date_week_number_35,arrival_date_day_of_month_1,arrival_date_day_of_month_10,arrival_date_day_of_month_11,...,reservation_status_date_22,reservation_status_date_23,reservation_status_date_24,reservation_status_date_25,reservation_status_date_26,reservation_status_date_27,reservation_status_date_28,reservation_status_date_29,reservation_status_date_30,reservation_status_date_31
0,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
traincols_list = list(cate_encoded_df.columns)

In [38]:
testcols_list = list(test_cate_encoded_df.columns)

In [39]:
misscols_list = list()
for column_name in traincols_list:
    if column_name in testcols_list:
        pass
    
    else:
        misscols_list.append(column_name)

In [40]:
misscols_list

['arrival_date_year_2015',
 'arrival_date_year_2016',
 'arrival_date_month_April',
 'arrival_date_month_December',
 'arrival_date_month_February',
 'arrival_date_month_January',
 'arrival_date_month_July',
 'arrival_date_month_June',
 'arrival_date_month_March',
 'arrival_date_month_May',
 'arrival_date_month_November',
 'arrival_date_month_October',
 'arrival_date_month_September',
 'arrival_date_week_number_1',
 'arrival_date_week_number_10',
 'arrival_date_week_number_11',
 'arrival_date_week_number_12',
 'arrival_date_week_number_13',
 'arrival_date_week_number_14',
 'arrival_date_week_number_15',
 'arrival_date_week_number_16',
 'arrival_date_week_number_17',
 'arrival_date_week_number_18',
 'arrival_date_week_number_19',
 'arrival_date_week_number_2',
 'arrival_date_week_number_20',
 'arrival_date_week_number_21',
 'arrival_date_week_number_22',
 'arrival_date_week_number_23',
 'arrival_date_week_number_24',
 'arrival_date_week_number_25',
 'arrival_date_week_number_26',
 'arriva

* 없어진 컬럼을 찾았다. 
* 이들은 범주형 변수이기 때문에 해당 인자가 없는 경우 0을 표시한다.
* 그러므로 해당 컬럼명으로 내용이 전부 0으로 되어있는 컬럼을 추가하면 된다.

In [41]:
for column in misscols_list:
    test_cate_encoded_df[column] = 0

In [42]:
# 확인
test_cate_encoded_df

,arrival_date_year_2017,arrival_date_month_August,arrival_date_week_number_31,arrival_date_week_number_32,arrival_date_week_number_33,arrival_date_week_number_34,arrival_date_week_number_35,arrival_date_day_of_month_1,arrival_date_day_of_month_10,arrival_date_day_of_month_11,...,arrival_date_week_number_53,arrival_date_week_number_6,arrival_date_week_number_7,arrival_date_week_number_8,arrival_date_week_number_9,market_segment_Undefined,distribution_channel_Undefined,reserved_room_type_L,assigned_room_type_L,yyyy_2015
0,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4920,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4921,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4922,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4923,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
# 아래 코드는 처음에 컬럼의 갯수가 더 많아진 문제가 발생했을 때 어떤 컬럼이 더 추가되었는지 보려고 하였던 코드 
# 바로 돌리면 돌아가지 않습니다.
# testcols_list 를 다시 불러들이는 작업 필요... 
# 현재는 실행하지 않아도 됩니다.

errcols_list = list()
for column_name in testcols_list:
    if column_name in traincols_list:
        pass
    
    else:
        errcols_list.append(column_name)

In [72]:
# country 정보가 자꾸 문제가 된다.
# 원래도 null 값으로 문제였기 때문에 처음에 모델 학습할 때 부터 빼기로 한다.
errcols_list

['country_LAO',
 'mm_01',
 'mm_02',
 'mm_03',
 'mm_04',
 'mm_05',
 'mm_06',
 'mm_07',
 'mm_08',
 'mm_09']

In [77]:
# 확인해보니 mm_1 과 같은 형식으로 들어갔어야 했다.
# 올라가서 바꿔두었다.
for i in cate_encoded_df.columns:
    print(i)

arrival_date_year_2015
arrival_date_year_2016
arrival_date_year_2017
arrival_date_month_April
arrival_date_month_August
arrival_date_month_December
arrival_date_month_February
arrival_date_month_January
arrival_date_month_July
arrival_date_month_June
arrival_date_month_March
arrival_date_month_May
arrival_date_month_November
arrival_date_month_October
arrival_date_month_September
arrival_date_week_number_1
arrival_date_week_number_10
arrival_date_week_number_11
arrival_date_week_number_12
arrival_date_week_number_13
arrival_date_week_number_14
arrival_date_week_number_15
arrival_date_week_number_16
arrival_date_week_number_17
arrival_date_week_number_18
arrival_date_week_number_19
arrival_date_week_number_2
arrival_date_week_number_20
arrival_date_week_number_21
arrival_date_week_number_22
arrival_date_week_number_23
arrival_date_week_number_24
arrival_date_week_number_25
arrival_date_week_number_26
arrival_date_week_number_27
arrival_date_week_number_28
arrival_date_week_number_29
arr

### X_data 만들기

In [46]:
test_X_data = pd.concat([test_numerical_df, test_cate_encoded_df], axis = 1)

In [48]:
test_X_data

,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_bookings_not_canceled,previous_cancellations,booking_changes,days_in_waiting_list,...,arrival_date_week_number_53,arrival_date_week_number_6,arrival_date_week_number_7,arrival_date_week_number_8,arrival_date_week_number_9,market_segment_Undefined,distribution_channel_Undefined,reserved_room_type_L,assigned_room_type_L,yyyy_2015
0,82,0,3,2,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,42,2,7,2,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,128,2,5,2,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,37,2,4,2,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,21,2,5,2,1.0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4920,23,2,5,2,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4921,102,2,5,3,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4922,34,2,5,2,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4923,109,2,5,2,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### X_data 정규화

In [49]:
X_data_copy = pd.concat([numerical_df, cate_encoded_df], axis = 1)

In [50]:
# 만들어두었던 Standard Scaler 를 쓰기 위해서는 컬럼의 갯수와 정렬상태가 같아야 한다고 한다. (에러메시지가..)
# 처음 상태를 알기 위해 기존 X_data를 가져와 데이터의 컬럼을 그대로 반영
# test_X_data에 들어간 컬럼명도 전부 다 같은 상태일 것이므로,
# 해당 코드가 정상적으로 동작하지 않는 경우 기존 X_data와 test_X_data의 컬럼이 서로 다른 오류가 있음을 의미

test_X_data = test_X_data[list(X_data_copy.columns)]

In [51]:
test_X_data = std_scl.transform(test_X_data)

In [186]:
test_X_data

array([[-0.49353713, -1.09163402,  0.02306799, ...,  0.        ,
         0.        ,  0.        ],
       [-0.8473362 ,  0.92839136,  2.12695439, ...,  0.        ,
         0.        ,  0.        ],
       [-0.0866682 ,  0.92839136,  1.07501119, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.91809601,  0.92839136,  1.07501119, ...,  0.        ,
         0.        ,  0.        ],
       [-0.25472276,  0.92839136,  1.07501119, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.59439501,  0.92839136,  2.12695439, ...,  0.        ,
         0.        ,  0.        ]])

In [52]:
test_X_data.shape

(4925, 207)

In [53]:
# 정확도 0.96
model.evaluate(test_X_data, test_y_data)

154/154 [==============================] - 0s 2ms/step - loss: 0.1354 - accuracy: 0.9604


[0.13542620837688446, 0.9604060649871826]

 * 이제 test_y_data의 실제값과 예측값이 얼마나 다른지 알아보아야 한다. 

In [55]:
pred = model.predict(test_X_data)

154/154 [==============================] - 0s 1ms/step


In [56]:
pred

array([[0.01657685],
       [0.3878439 ],
       [0.10614104],
       ...,
       [0.00390925],
       [0.04873647],
       [0.0394027 ]], dtype=float32)

In [61]:
pred_test_X_data = list()

for i in pred:
    
    if float(i) > 0.5:
        pred_test_X_data.append(1)
        
    else:
        pred_test_X_data.append(0)

In [85]:
new_df = pd.DataFrame()

In [86]:
new_df['is_canceled'] = test_y_data
new_df['predict'] = pred_test_X_data

In [88]:
new_df.reset_index(inplace = True)

In [89]:
new_df

,index,is_canceled,predict
0,0,0,0
1,1,0,0
2,2,0,0
3,3,0,0
4,4,0,0
...,...,...,...
4920,4920,0,0
4921,4921,0,0
4922,4922,0,0
4923,4923,0,0


In [90]:
S3Client.upload_s3_df(new_df, file_name="inference/student1/test_hotel.parquet")

True

In [106]:
df = S3Client.get_s3_parquet_df("inference/student1/test_hotel.parquet")
df

,index,is_canceled,predict
0,0,0,0
1,1,0,0
2,2,0,0
3,3,0,0
4,4,0,0
...,...,...,...
4920,4920,0,0
4921,4921,0,0
4922,4922,0,0
4923,4923,0,0


### AUC 구하기?

In [109]:
f1 = f1_score(new_df['is_canceled'], new_df['predict'])

In [110]:
f1

0.9446494464944648

In [112]:
roc_auc = roc_auc_score(new_df['is_canceled'], new_df['predict'])
print("ROC AUC Score: {}".format(roc_auc))

ROC AUC Score: 0.9512343728524406


In [114]:
# 아래 방법으로 confusion_matrix를 그려 확인해볼 수 있다.
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(new_df['is_canceled'], new_df['predict'])
cm

array([[3066,   43],
       [ 152, 1664]])

In [116]:
cmtx = pd.DataFrame(cm, index = ['현실틀림', '현실맞음'], columns = ['틀리다고예측', '맞다고예측'])

In [117]:
cmtx

,틀리다고예측,맞다고예측
현실틀림,3066,43
현실맞음,152,1664
